## Seq2Seq
    generally seq2seq algorithm is used, when input sequences and output sequences have different lengths (e.g. machine translation) and the entire input sequence is required in order to start predicting the target. 
Here we are predicting character to character representation.

*For more INFO, check here: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

*For language translation datasets, click here: http://www.manythings.org/anki/

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,LSTM,Input
import numpy as np

In [2]:
batch_size =64
epochs = 100
latent_dim = 256  #output dimensionality of the encoding space.
num_samples = 10000  #number of samples to train on.
data_path = "datasets/Machine Language Translation datasets/English - French language dataset.txt"

In [3]:
#Vectorize the data
input_texts = []  
target_texts = []  
input_characters = set()  #input_text
target_characters = set()  #output text
with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    
    #we use 'tab' as start sequence character for the targets, and '\n' as end sequence character
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
lines[:3]

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)']

In [5]:
#input_characters

In [6]:
#target_characters

In [36]:
target_texts[:5]

['\tVa !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n']

In [8]:
input_texts[:5]

['Go.', 'Hi.', 'Hi.', 'Run!', 'Run!']

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_len = max([len(txt) for txt in input_texts])
max_decoder_seq_len = max([len(txt) for txt in target_texts])

In [18]:
#getting max_encoder_seq_len's text
text = np.argmax([len(txt) for txt in input_texts])
input_texts[text]

'A boat capsized.'

In [5]:
print('no. of samples for training: ', len(input_texts))
print('no. of unique input tokens(Eng):', num_encoder_tokens)
print('no. of unique output tokens(French):', num_decoder_tokens)
print('Max sequence of input length(Eng):', max_encoder_seq_len)
print('Max sequence of output length(Fre):', max_decoder_seq_len)

no. of samples for training:  10000
no. of unique input tokens(Eng): 71
no. of unique output tokens(French): 92
Max sequence of input length(Eng): 16
Max sequence of output length(Fre): 59


In [5]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])

target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])

In [15]:
input_token_index, target_token_index

({' ': 0,
  '!': 1,
  '$': 2,
  '%': 3,
  '&': 4,
  "'": 5,
  ',': 6,
  '-': 7,
  '.': 8,
  '0': 9,
  '1': 10,
  '2': 11,
  '3': 12,
  '5': 13,
  '6': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  'é': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  '(': 7,
  ')': 8,
  ',': 9,
  '-': 10,
  '.': 11,
  '0': 12,
  '1': 13,
  '2': 14,
  '3': 15,
  '5': 16,
  '8': 17,
  '9': 18,
  ':': 19,
  '?': 20,
  'A'

In [6]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_len, num_encoder_tokens), dtype = 'float32')

decoder_input_data =  np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype = 'float32')

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype = 'float32')

In [7]:
encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((10000, 16, 71), (10000, 59, 92), (10000, 59, 92))

#### applying one-hot representiation

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1.  #replacing all with 1. with space character indexes..
    
    for t, char in enumerate(target_text):
        #decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i,t, target_token_index[char]] = 1.
        
        if t > 0:
            #decoder_target_data will be ahead by one timestep and will not include the start character('\t')
            decoder_target_data[i,t-1, target_token_index[char]] = 1.
            
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [9]:
decoder_target_data[0,1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [10]:
input_texts[0]


'Go.'

In [11]:
#RNN ENCODER 
#Define an input sequence and preprocess it.
encoder_inputs = Input(shape =(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True) 
encoder_outputs, state_h, state_c = encoder(encoder_inputs)  #return_state = True -- gives the output along with state_h and state_c
#we discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h, state_c]


In [16]:
print(state_h)

Tensor("lstm/Identity_1:0", shape=(None, 256), dtype=float32)


In [13]:
#RNN DECODER
#setup the decoder, using 'encoder_states' as initial state
decoder_inputs = Input(shape = (None, num_decoder_tokens))
#we set up out decoder to return full output sequences and to return inernal states as well.
#we don't use the return states in the training model, but we will use them in prediction.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## NOTE:
    encoder LSTM doesn't pass directly it's outputs as inputs to the decoder LSTM instead the decoder uses the final hidden and cell states as the initial state for decoder.
    
    Also note that Decoder LSTM pass the whole sequence of hidden states to the Dense (output layer) for output, instead of the final  hidden and cell state.

In [14]:
#Define the model that will turn
#'encoder_input_data' and 'decoder_input_data' into 'decoder_target_data'
model = Model(inputs = [encoder_inputs, decoder_inputs], outputs = decoder_outputs)

model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy', metrics =['accuracy'])

#### Graphical representation of encoder - decoder model :
https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/10/Graph-of-Encoder-Decoder-Model-For-Training.png

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 92)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  357376      input_2[0][0]                    
                                                                 lstm[0][1]                   

### Training Begins

In [13]:
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, batch_size = batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 26s 208ms/step - loss: 1.1781 - accuracy: 0.7235 - val_loss: 1.1070 - val_accuracy: 0.6977
Epoch 2/100
125/125 [==============================] - 26s 212ms/step - loss: 0.8515 - accuracy: 0.7707 - val_loss: 0.8366 - val_accuracy: 0.7717
Epoch 3/100
125/125 [==============================] - 26s 212ms/step - loss: 0.6770 - accuracy: 0.8081 - val_loss: 0.7280 - val_accuracy: 0.7920
Epoch 4/100
125/125 [==============================] - 26s 211ms/step - loss: 0.5929 - accuracy: 0.8273 - val_loss: 0.6481 - val_accuracy: 0.8113
Epoch 5/100
125/125 [==============================] - 27s 214ms/step - loss: 0.5447 - accuracy: 0.8410 - val_loss: 0.6080 - val_accuracy: 0.8208
Epoch 6/100
125/125 [==============================] - 26s 209ms/step - loss: 0.5097 - accuracy: 0.8503 - val_loss: 0.5784 - val_accuracy: 0.8288
Epoch 7/100
125/125 [==============================] - 26s 208ms/step - loss: 0.4814 - accuracy: 0.8582 - val_loss: 0.5599 -

Epoch 57/100
125/125 [==============================] - 25s 203ms/step - loss: 0.1108 - accuracy: 0.9653 - val_loss: 0.5526 - val_accuracy: 0.8750
Epoch 58/100
125/125 [==============================] - 25s 204ms/step - loss: 0.1088 - accuracy: 0.9657 - val_loss: 0.5622 - val_accuracy: 0.8747
Epoch 59/100
125/125 [==============================] - 26s 204ms/step - loss: 0.1071 - accuracy: 0.9661 - val_loss: 0.5608 - val_accuracy: 0.8740
Epoch 60/100
125/125 [==============================] - 25s 203ms/step - loss: 0.1047 - accuracy: 0.9669 - val_loss: 0.5662 - val_accuracy: 0.8746
Epoch 61/100
125/125 [==============================] - 26s 207ms/step - loss: 0.1032 - accuracy: 0.9675 - val_loss: 0.5690 - val_accuracy: 0.8748
Epoch 62/100
125/125 [==============================] - 25s 204ms/step - loss: 0.1010 - accuracy: 0.9681 - val_loss: 0.5688 - val_accuracy: 0.8756
Epoch 63/100
125/125 [==============================] - 25s 202ms/step - loss: 0.0991 - accuracy: 0.9685 - val_loss: 0

### saving our model

In [20]:
model.save('saved models/English-French Translation.h5')

### Loading model

In [15]:
from tensorflow.keras.models import load_model

In [16]:
MODEL = load_model('saved models/English-French Translation.h5')
MODEL.compile(optimizer = 'rmsprop', loss='categorical_crossentropy', metrics =['accuracy'])

In [17]:
MODEL.layers

## Execution TIME

In [18]:
#Defining Encoder layer
encoder = MODEL.layers[2]
encoder_inputs = MODEL.layers[0].input
encoder_outputs, state_h, state_c = MODEL.layers[2].output
encoder_states = [state_h, state_c]

encoder_model = Model(inputs= encoder_inputs, outputs= encoder_states)
#it takes numpy array input and returns output in the form of list of 2 elements.

#### Graph of Encoder Model For Inference:
https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/10/Graph-of-Encoder-Model-For-Inference.png

In [16]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 71)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 335872    
Total params: 335,872
Trainable params: 335,872
Non-trainable params: 0
_________________________________________________________________


In [19]:
#Defining Decoder layer

decoder_inputs = MODEL.layers[1].input
decoder_lstm = MODEL.layers[3]
decoder_dense = MODEL.layers[-1]

decoder_state_input_h = Input(shape= (latent_dim,))
decoder_state_input_c = Input(shape= (latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,
                                                 initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs=decoder_dense(decoder_outputs)

decoder_model = Model(inputs = [decoder_inputs] + decoder_states_inputs,
                     outputs = [decoder_outputs] + decoder_states)
# it takes and returns list of 3 elements..

#### Graph of Decoder Model For Inference:
https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/10/Graph-of-Decoder-Model-For-Inference.png

In [68]:
decoder_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 92)]   0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  357376      input_2[0][0]                    
                                                                 input_5[0][0]              

In [20]:
#reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i,char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i,char) for char, i in target_token_index.items())

In [21]:
def decode_sequence(input_seq):
    
    states_value = encoder_model.predict(input_seq)# this model outputs only list of encoder states..
    
    target_seq = np.zeros((1,1, num_decoder_tokens))
    
    target_seq[0,0, target_token_index['\t']] = 1.
    
  
    stop_condition=False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
       
        sampled_token_index = np.argmax(output_tokens[0,-1,:])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True
            
        #update the target sequence (of length 1)
        target_seq = np.zeros((1,1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index] =1.
        
        #update states
        states_value = [h,c]
        
    return decoded_sentence

In [22]:
for seq_index in range(3):
    #take the sequence (part of the training set) for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index +1] #since encoder takes 3-D input..
    #above line can also be replaced as input_seq = encoder_input_data[seq_index].reshape((1,) + encoder_input_data[seq_index])
    
    decoded_sentence = decode_sequence(input_seq)
    print('---------------------------')
    print('Input sentence: ', input_texts[seq_index])
    print('Decoded sentence: ', decoded_sentence)

---------------------------
Input sentence:  Go.
Decoded sentence:  Va !

---------------------------
Input sentence:  Hi.
Decoded sentence:  Salut.

---------------------------
Input sentence:  Hi.
Decoded sentence:  Salut.

